In [6]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 30.5 MB/s eta 0:00:00


In [13]:
import os

for root, dirs, files in os.walk(rupankarmajumdar_disaster_response_object_detection_dataset_path):
    print(root)
    break  # just show top-level


/root/.cache/kagglehub/datasets/rupankarmajumdar/disaster-response-object-detection-dataset/versions/1


In [1]:
import kagglehub
rupankarmajumdar_disaster_response_object_detection_dataset_path = kagglehub.dataset_download('rupankarmajumdar/disaster-response-object-detection-dataset')

print('Data source import complete.')


100%|██████████| 977M/977M [00:07<00:00, 134MB/s]

Extracting files...


Data source import complete.


In [3]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input/disaster-response-object-detection-dataset'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


In [4]:
# ============================================================
# YOLO Dataset Visualization
# ============================================================

import os
import cv2
import random
import matplotlib.pyplot as plt
from pathlib import Path

# ------------------------------------------------------------
# 1️⃣ Load Dataset Config (already a dict, no need for open)
# ------------------------------------------------------------
data_cfg = {
    "train": "/kaggle/input/disaster-response-object-detection-dataset/train/images",
    "val": "/kaggle/input/disaster-response-object-detection-dataset/val/images",
    "test": "/kaggle/input/disaster-response-object-detection-dataset/test/images",
    "nc": 6,
    "names": [
        "person", "fire", "smoke", "small_vehicle", "large_vehicle", "two_wheeler"
    ]
}

class_names = data_cfg["names"]
splits = ["train", "val", "test"]

print(f"✅ Dataset splits detected: {splits}")
print(f"✅ Class names: {class_names}")

# ------------------------------------------------------------
# 2️⃣ Visualization Function
# ------------------------------------------------------------
def visualize_yolo_split(split_name, num_images=3):
    """
    Visualize YOLO-format annotations from dataset split.
    """
    image_dir = Path(data_cfg[split_name])
    # Labels assumed in a sibling folder 'labels'
    label_dir = image_dir.parent / "labels"

    # Check if labels exist
    if not label_dir.exists():
        print(f"⚠️ Label directory not found: {label_dir}")
        return

    # Get image files
    img_files = list(image_dir.glob("*.jpg")) + list(image_dir.glob("*.png"))
    if len(img_files) == 0:
        print(f"⚠️ No images found in {image_dir}")
        return

    random.shuffle(img_files)
    img_files = img_files[:num_images]

    print(f"\n📸 Showing {num_images} sample images from '{split_name}' split")

    for img_path in img_files:
        label_path = label_dir / f"{img_path.stem}.txt"
        img = cv2.imread(str(img_path))
        if img is None:
            continue
        h, w = img.shape[:2]

        # Draw YOLO labels if exists
        if label_path.exists():
            with open(label_path, "r") as f:
                lines = f.readlines()
                for line in lines:
                    cls, x, y, bw, bh = map(float, line.strip().split())
                    cls = int(cls)
                    x1 = int((x - bw / 2) * w)
                    y1 = int((y - bh / 2) * h)
                    x2 = int((x + bw / 2) * w)
                    y2 = int((y + bh / 2) * h)
                    cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
                    cv2.putText(img, class_names[cls], (x1, y1 - 5),
                                cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 0), 2)

        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(8, 8))
        plt.imshow(img_rgb)
        plt.axis("off")
        plt.title(f"{split_name.upper()} | {img_path.name}")
        plt.show()

# ------------------------------------------------------------
# 3️⃣ Visualize Each Split
# ------------------------------------------------------------
visualize_yolo_split("train", num_images=4)
visualize_yolo_split("val", num_images=4)
visualize_yolo_split("test", num_images=4)

✅ Dataset splits detected: ['train', 'val', 'test']
✅ Class names: ['person', 'fire', 'smoke', 'small_vehicle', 'large_vehicle', 'two_wheeler']
⚠️ Label directory not found: /kaggle/input/disaster-response-object-detection-dataset/train/labels
⚠️ Label directory not found: /kaggle/input/disaster-response-object-detection-dataset/val/labels
⚠️ Label directory not found: /kaggle/input/disaster-response-object-detection-dataset/test/labels


In [14]:
from ultralytics import YOLO
import cv2
import random
import yaml
import matplotlib.pyplot as plt
from pathlib import Path

# ------------------------------------------------------------
# 1️⃣ Load Dataset YAML
# ------------------------------------------------------------
yaml_path = Path("data.yaml")

with open(yaml_path, "r") as f:
    data_cfg = yaml.safe_load(f)

print("✅ Dataset loaded successfully:")
print(yaml.dump(data_cfg, sort_keys=False))

✅ Dataset loaded successfully:
train: /root/.cache/kagglehub/datasets/rupankarmajumdar/disaster-response-object-detection-dataset/versions/1/train/images
val: /root/.cache/kagglehub/datasets/rupankarmajumdar/disaster-response-object-detection-dataset/versions/1/val/images
test: /root/.cache/kagglehub/datasets/rupankarmajumdar/disaster-response-object-detection-dataset/versions/1/test/images
nc: 6
names:
- person
- fire
- smoke
- small_vehicle
- large_vehicle
- two_wheeler



In [17]:
# ------------------------------------------------------------
# 2️⃣ Train YOLOv10n Model
# ------------------------------------------------------------
model = YOLO("yolov10s.pt")  # pre-trained model

results = model.train(
    data=str(yaml_path),
    epochs=10,
    imgsz=640,
    batch=16,
    project="/kaggle/working/disaster_response_object_yolov10_train",
    name="yolov10_disaster",
    device=0,
    workers=2,
    exist_ok=True
)

print("\n✅ Training complete. Best model saved in:")
print(model.ckpt_path)

Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov10s.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolov10_disaster, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretra

In [18]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [19]:
!mkdir -p "/content/drive/MyDrive/YOLO_Models"
!cp /kaggle/working/disaster_response_object_yolov10_train/yolov10_disaster/weights/best.pt "/content/drive/MyDrive/YOLO_Models/best_disaster_yolov10s.pt"


In [20]:
# ------------------------------------------------------------
# 3️⃣ Evaluate on Validation and Test Sets
# ------------------------------------------------------------
print("\n📊 Evaluating on validation set...")
val_metrics = model.val(data=str(yaml_path), split="val")
print("\nValidation Results:")
#print(val_metrics)

print("\n📊 Evaluating on test set...")
test_metrics = model.val(data=str(yaml_path), split="test")
print("\nTest Results:")
#print(test_metrics)


📊 Evaluating on validation set...
Ultralytics 8.3.225 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
YOLOv10s summary (fused): 106 layers, 7,220,322 parameters, 0 gradients, 21.4 GFLOPs
val: Fast image access ✅ (ping: 0.0±0.0 ms, read: 1070.5±784.4 MB/s, size: 60.6 KB)
val: Scanning /root/.cache/kagglehub/datasets/rupankarmajumdar/disaster-response-object-detection-dataset/versions/1/val/labels.cache... 1556 images, 24 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 1556/1556 782.7Kit/s 0.0s
val: /root/.cache/kagglehub/datasets/rupankarmajumdar/disaster-response-object-detection-dataset/versions/1/val/images/1058.jpg: 1 duplicate labels removed
val: /root/.cache/kagglehub/datasets/rupankarmajumdar/disaster-response-object-detection-dataset/versions/1/val/images/1283.jpg: 4 duplicate labels removed
val: /root/.cache/kagglehub/datasets/rupankarmajumdar/disaster-response-object-detection-dataset/versions/1/val/images/1373.jpg: 2 duplicate labels removed
val: /root/.cache/kaggle